# Multivariate adaptive shrinkage (MASH) analysis of GTEx data

This notebook contains code used to generate results for the Urbut *et al* (2017) manuscript.

Important notes:

1. Although you may open this notebook in Jupyter, you should not step through the code sequentially as you would in a typical Jupyter notebook; this is because the code in this notebook is meant to be run using the [Script of Scripts (SoS)](https://github.com/vatlab/SoS)
framework.

2. This notebook is not only used to reproduce Urbut 2017, but with the `--data` option you can provide your own data-set and perform MASH analysis. Here the input data have already been converted from the original eQTL summary statistics (downloaded from the GTEx Portal) to a format convenient for analysis in MASH. For details on this step, see the README and the `fastqtl_to_mash.ipynb` notebook. 

3. In this notebook, we did not apply the inference to all the gene-snp pairs. Rather we focused on the "top" gene-snp pairs as a demonstration. It should be straightforward to configure the Posterior computatoin step to work on all gene-snp pairs instead. But we'd recommand using the `mashr` workflow which is a lot faster (see below).

4. We have improved MASH implementation since the paper was submitted. See section "Analysis with `mashr` package" for the `mashr` workflow, and section "Run this notebook" about how to trigger this new workflow.

## Run MASH as implemented in Urbut 2017
For repeated runs it might be easier to execute from commandline instead of in Jupyter:

```bash
sos run workflows/gtex6_mash_analysis.ipynb # --data ... --cwd ...
```

The notebook runs default setting, ie, all the analysis steps. To view all available analysis:

```bash
sos run workflows/gtex6_mash_analysis.ipynb -h
```

Additionally I run it for dataset after LD pruning (for LD related discussion in supplemental information):

```bash
sos run workflows/gtex6_mash_analysis.ipynb --data data/MatrixEQTLSumStats.Portable.ld2.Z.rds
```

The outcome of this notebook should be found under `./gtex6_workflow_output` folder (can be configured), with the following output:

```
gtex6_mash_analysis.html
MatrixEQTLSumStats.Portable.Z.sfa.rds.log
MatrixEQTLSumStats.Portable.Z.sfa.rds
MatrixEQTLSumStats.Portable.Z.coved.K3.P3.rds
MatrixEQTLSumStats.Portable.Z.coved.K3.P3.lite.single.expanded.rds
MatrixEQTLSumStats.Portable.Z.coved.K3.P3.lite.single.expanded.pihat.rds
MatrixEQTLSumStats.Portable.Z.coved.K3.P3.lite.single.expanded.loglik.rds
MatrixEQTLSumStats.Portable.Z.coved.K3.P3.lite.single.expanded.posterior.rds
```

We keep track of results from every MASH step, though the inference of interest should be found in the `*.posterior.rds` file generated at the end of the pipeline.

### Analysis Steps

The pipeline automatically executes the following:

+ Compute a sparse factorization of the (centered) z-scores using the
  [SFA software](http://stephenslab.uchicago.edu/software.html#sfa),
  with K = 5 factors, and save the factors in an `.rds` file. This
  will be used to construct the mixture-of-multivariate normals
  prior. This step is labeled `sfa`, and should only take a few
  minutes to run.

+ Compute additional "data-driven" prior matrices by computing a
  singular value decomposition of the (centered) z-scores and low-rank
  approximations to the empirical covariance matrices. Most of the
  work in this step involves running the Extreme Deconvolution
  method. The outcome of running the Extreme Deconvolution method is
  saved to a new `.rds` file. This step is labeled `mash-paper_1` and
  may take several hours to run (in one run on a MacBook Pro with
  a 3.5 GHz Intel Core i7, it took over 6 hours to complete).

+ A final collection of "canonical" and single-rank prior matrices
  based on SFA and the "BMAlite" models of Flutre *et al*
  (2013). These matrices are again written to another `.rds` file. This
  step is labeled `mash-paper_2`, and should take at most a minute to
  run.

+ The `mash-paper_3` step fits the MASH ("multivariate adaptive
  shrinkage") model to the GTEx data (the centered z-scores); the
  model parameters estimated in this fitting step are the weights of
  the multivariate normal mixture. The outputs from this step are the
  estimated mixture weights and the conditional likelihood
  matrix. These two outputs are saved to two separate `.rds` files.
  This step is expected to take at most a few hours to complete.

+ The `mash-paper_4` step computes posterior statistics using the
  fitted MASH model from the previous step. These posterior statistics
  are summarized and visualized in subsequent analyses. The posterior
  statistics are saved to another `.rds` file. This step is expected
  to take a few hours to complete.


### A faster implementation via package `mashr`

See section "Analysis with `mashr` package" and the `mashr` workflow for details. To use this workflow, add `--algorithm mashr` to the run commands above.

If you run from the docker image `gaow/mash-paper` you need to put a copy of [MOSEK license file](https://www.mosek.com/products/academic-licenses) to `<workdir>/mosek.lic` (ie, `gtex6_workflow_output/mosek.lic` if you did not change any settings below).

### The prototype (default) MASH implementation

Some notes about the code used for the `mash` workflow analysis (some available from the `mash` workflow, some in the downloaded `mash_script_download` code):
    
    1. `deconvolution.em.with.bovy(...)`

produces an object with the denoised matrices for feeding into the
*mash* covariance code. The *factor.mat* and *lambda.mat* called
within have been produced by SFA and are single rank factors and
loadings approximating the empirical covariance.

    2. `compute.hm.covmat.all.max.step(...)$covmat` 

produces a list of covariance matrices entitled *covmat"A".rds* upon
which to base the mixture of multivariate normals.

    3. `compute.hm.train.log.lik(...)`

computes the HM weights on training datauses the set of randomly chosen genes to train our model and produces
a matrix of likelihoods and corresponding hierarchical weights, as well as the mixture proportions.

    4. `total.quant.per.snp(...)`

produces files containing the posterior means, upper and lower
tail probabilities, null probabilites, and lfsr for all J genes across
44 conditions.

In [1]:
[global]
parameter: cwd = path('./gtex6_workflow_output')
parameter: data = path("data/MatrixEQTLSumStats.Portable.Z.rds")
# path configured to /opt folder outside $HOME, to make it easier to use with `docker`
parameter: mash_src = file_target("/opt/mash-paper/main.R")
parameter: sfa_exe = file_target("/opt/sfa/bin/sfa_linux")
# parameters for mashr analysis
parameter: algorithm = 'paper'
parameter: vhat = 1
mosek_license = file_target(f"{cwd:a}/mosek.lic")
sfa_data = file_target(f"{cwd:a}/{data:bn}.sfa.rds")
flash_data = file_target(f"{cwd:a}/{data:bn}.flash.rds")

Workflow can only be executed with magic %run or %sosrun.

## Covariance pattern discovery
This obtains covariance matrices, ie, the priors, for `mash` model.

### SFA
We analyze data with SFA. The cell below downloads SFA software and run it on data with rank `K = 5`.

In [2]:
[sfa_download: provides = sfa_exe]
# Download / install SFA (no need if running from docker `gaow/mash-paper`)
download: decompress = True, dest_dir = f'{sfa_exe:ad}'
    http://stephenslab.uchicago.edu/assets/software/sfa/sfa1.0.tar.gz

[sfa: provides = sfa_data]
# Perform SFA analysis (time estimate: <1min)
depends: sfa_exe
K = 5
tmpfile = path(f"{cwd:a}/{data:bn}.max.txt")
input: f"{data:a}"
output: sfa_data
R: expand = "${ }", stdout = f"{_output:n}.log", workdir = cwd
    z = readRDS(${_input:r})$test.z
    write.table(z, ${tmpfile:r}, col.names=F,row.names=F)
    cmd = paste0('${sfa_exe} -gen ${tmpfile} -g ', dim(z)[1], ' -n ', dim(z)[2], 
                 ' -k ${K} -iter 50 -rand 999 -o ${_output:bn}')
    system(cmd)
    saveRDS(list(F = read.table("${_output:n}_F.out"),
                lambda = read.table("${_output:n}_lambda.out"),
                sigma2 = read.table("${_output:n}_sigma2.out"),
                alpha = read.table("${_output:n}_alpha.out")), ${_output:r})
bash: workdir = cwd, expand = '${ }'
    rm -f *{_F.out,_lambda.out,_sigma2.out,_alpha.out} ${tmpfile}
    rm -r output

Workflow can only be executed with magic %run or %sosrun.

###  Create and refine multi-rank covariance matrices
Here we create 3 covariance matrices:

* SFA (rank 5, previously computed)
* SVD (rank 3, to be computed)
* Empirical covariance

and apply [Extreme Deconvolution](https://github.com/jobovy/extreme-deconvolution) to refine the matrices. We observed that Extreme Deconvolution perserves rank.

In [3]:
[mash_scripts_download: provides = mash_src]
# Download / install MASH scripts (no need if running from docker `gaow/mash-paper`)
output: mash_src
download: decompress = True, dest_dir = cwd
    https://github.com/stephenslab/mashr-paper/archive/v0.2-1.zip
bash: expand = True, workdir = cwd
    mkdir -p {mash_src:ad}
    cp mashr-paper-0.2-1/R/* {mash_src:ad} && rm -rf mashr-paper-0.2-1

[mash-paper_1: shared = {'mash_input': '_input'}]
# Compute data-driven prior matrices 
# (time estimate: 40min to 4hrs depending on the machine power)
depends: R_library("ExtremeDeconvolution"), mash_src, sfa_data
K = 3
P = 3
input: f"{data:a}"
output: f"{cwd:a}/{data:bn}.coved.K{K}.P{P}.rds"
R: expand = "${ }", workdir = cwd
    library(ExtremeDeconvolution)
    script.source = sapply(list.files(${mash_src:dar}, pattern = "*.R", full.names=TRUE), source, .GlobalEnv)
    z.stat = readRDS(${_input:r})$test.z
    sfa = readRDS(${sfa_data:r})
    s.j = matrix(rep(1,ncol(z.stat)*nrow(z.stat)),ncol=ncol(z.stat),nrow=nrow(z.stat))
    res = deconvolution.em.with.bovy(z.stat, 
                                      as.matrix(sfa$F), 
                                      s.j, 
                                      as.matrix(sfa$lambda),
                                      K = ${K}, P = ${P})
    saveRDS(res, ${_output:r})

Workflow can only be executed with magic %run or %sosrun.

### Add in canonical and single-rank covariance matrices

Now additionally we include 2 other types of covariance matrices:
* canonical configurations (aka `bmalite`)
* single rank SFA

We also expand the list of matrices by grid. At the end of this step (cell below) we are ready to fit the mash model.

In [4]:
[mash-paper_2: shared = {'prior_matrices': '_output'}]
# Add in canonical configurations and single rank SFA priors (time estimate: <1min)
depends: sos_variable('mash_input'), sfa_data
output: f"{_input:n}.lite.single.expanded.rds"
R: expand = "${ }", workdir = cwd
    script.source = sapply(list.files(${mash_src:dar}, pattern = "*.R", full.names=TRUE), source, .GlobalEnv)
    z.stat = readRDS(${mash_input:r})$test.z
    rownames(z.stat) = NULL
    colnames(z.stat) = NULL
    s.j = matrix(rep(1,ncol(z.stat)*nrow(z.stat)),ncol=ncol(z.stat),nrow=nrow(z.stat))
    sfa = readRDS(${sfa_data:r})
    res = compute.hm.covmat.all.max.step(b.hat=z.stat,se.hat=s.j,
                                          t.stat=z.stat,Q=5,
                                          lambda.mat=as.matrix(sfa$lambda),
                                          A='.${_input:bn}.remove_before_rerun',
                                          factor.mat=as.matrix(sfa$F),
                                          max.step=readRDS(${_input:r}),
                                          zero=TRUE)
    saveRDS(res$covmat, ${_output:r})

bash: workdir = cwd, expand = True
    rm -f *.{_input:bn}.remove_before_rerun.*

Workflow can only be executed with magic %run or %sosrun.

## Fit MASH mixture model
Using a training set, the cell below computes the weights for input covariance matrices (priors) in MASH mixture. The output contains matrix of log-likelihoods as well as weights learned from the hierarchical model.

In [5]:
[mash-paper_3]
# Fit MASH mixture model (time estimate: ~2.5hr)
depends: sos_variable('mash_input'), R_library("SQUAREM")
output: f"{_input:n}.V{vhat}.pihat.rds", f"{_input:n}.V{vhat}.loglik.rds"
R: expand = "${ }", workdir = cwd
    library("SQUAREM")
    script.source = sapply(list.files(${mash_src:dar}, pattern = "*.R", full.names=TRUE), source, .GlobalEnv)
    dat = readRDS(${mash_input:r})
    covmat = readRDS(${_input:r})
    train.z = as.matrix(dat$train.z)
    rownames(train.z) = NULL
    colnames(train.z) = NULL
    train.v = matrix(rep(1,ncol(train.z)*nrow(train.z)),ncol=ncol(train.z),nrow=nrow(train.z))
    res = compute.hm.train.log.lik.pen.vmat(train.b=train.z,
                                            covmat=covmat,
                                            cormat=${"dat$vhat" if vhat else "diag(nrow(dat$vhat))"},
                                            A='.${_output[0]:bnn}.remove_before_rerun', 
                                            pen=TRUE,
                                            train.s=train.v)
    saveRDS(res$pis, ${_output[0]:r})
    saveRDS(res$lik.mat, ${_output[1]:r})

bash: workdir = cwd, expand = True
    rm -f *.{_output[0]:bnn}.remove_before_rerun.*

Workflow can only be executed with magic %run or %sosrun.

## Posterior inference
Applying hyperparameters learned from the training set to the test set, the cell below computes posterior quantities. 

In [6]:
[mash-paper_4]
# Posterior inference on the "top" set of gene-snp pairs 
# (time estimate: ~5hr on single thread)
depends: sos_variable('mash_input'), sos_variable('prior_matrices')
output: f"{_input[0]:nn}.posterior.rds"
R: expand = "${ }", workdir = cwd
    script.source = sapply(list.files(${mash_src:dar}, pattern = "*.R", full.names=TRUE), source, .GlobalEnv)
    dat = readRDS(${mash_input:r})
    z.stat = dat$test.z
    s.j = matrix(rep(1,ncol(z.stat)*nrow(z.stat)),ncol=ncol(z.stat),nrow=nrow(z.stat))
    pis = readRDS(${_input[0]:r})$pihat
    covmat = readRDS(${prior_matrices:r})
    res = lapply(seq(1:nrow(z.stat)), function(j){
          total.quant.per.snp.with.vmat(j=j, 
                                      covmat=covmat, 
                                      b.gp.hat=z.stat,
                                      se.gp.hat=s.j, 
                                      cormat=${"dat$vhat" if vhat else "diag(nrow(dat$vhat))"},
                                      pis=pis, 
                                      A='', 
                                      checkpoint=TRUE)})
    # data formatting.
    out = do.call(Map, c(f = rbind, res))
    saveRDS(out, ${_output:r})

Workflow can only be executed with magic %run or %sosrun.

Now MASH analysis is complete. I will use separate vignettes to summarize, plot and visualize the result of analysis.

## Analysis with `mashr` package

Since the submission of Urbut 2017 we have improved implementation of MASH algorithm to produce an R package, [`mashr`](https://github.com/stephenslab/mashr). Major improvements compared to Urbut 2017 are:

1. Faster computation of likelihood and posterior matrices via matrix algebra tricks and a C++ implementation.
2. Faster computation of MASH mixture via convex optimization.
3. Add in `FLASH` method for prior covariance matrices.

Below is the same workflow as `mash` workflow previously discussed, but implemented with `mashr`.

### FLASH analysis

In [7]:
[flash: provides = flash_data]
# Perform FLASH analysis (time estimate: 20min)
depends: R_library("mashr@stephenslab/flashr"), R_library('mclust'), R_library('plyr')
K = 5 # flash factors
input: f"{data:a}"
output: flash_data
R: expand = "${ }", workdir = cwd
    get_flash_prior = function(test.z, K = ${K}, factor_plot = "${_output:n}.factors.pdf") {
        Z.center = apply(test.z, 2, function(x) x - mean(x))
        flash.data = flashr::flash_set_data(Z.center)
        fmodel = flashr::flash(flash.data, greedy = TRUE, backfit = TRUE)
        factors = flashr::flash_get_ldf(fmodel)$f
        row.names(factors) = colnames(test.z)
        pve.order = order(flashr::flash_get_pve(fmodel), decreasing = TRUE)
        if (!is.null(factor_plot)) {
            pdf(factor_plot)
            #par(mar=c(.5,.5,.5,.5))
            #par(mfrow=c(ceiling(length(pve.order) / 2), 2))
            for(i in pve.order){
              barplot(factors[,i], main=paste0('Factor ',i, ' pve= ', round(flashr::flash_get_pve(fmodel)[i],3)), las=2, cex.names = 0.7)
            }
            dev.off()
          }
        # flash on the loading
        loading = fmodel$EL[,1:K]
        colnames(loading) = paste0('Factor',seq(1,K))
        flash.loading = flashr::flash_set_data(loading)
        flmodel = flashr::flash(flash.loading, greedy = TRUE, backfit = TRUE)
        # Cluster loadings
        library(mclust)
        mod = Mclust(loading)
        U_list = plyr::alply(mod$parameters$variance$sigma,3)
        mu_list = plyr::alply(mod$parameters$mean,2)
        ll = list()
        for (i in 1:length(U_list)){
          ll[[i]] = U_list[[i]] + mu_list[[i]] %*% t(mu_list[[i]])
        }
        Factors = fmodel$EF[,1:5]
        U.loading = lapply(ll, function(U){Factors %*% (U %*% t(Factors))})
        names(U.loading) = paste0('Load', K, "_", (1:length(U.loading)))
        # rank 1
        Flash_res = flashr::flash_get_lf(fmodel)
        U.flash = c(mashr::cov_from_factors(t(as.matrix(factors)), "Flash"),
                    list("tFlash" = t(Flash_res) %*% Flash_res / nrow(Z.center)))
        return(list(U.loading = U.loading, U.flash = U.flash))
    }
    #
    res = get_flash_prior(readRDS(${_input:r})$test.z)
    saveRDS(res, ${_output:r})

Workflow can only be executed with magic %run or %sosrun.

### MASH analysis

In [8]:
[mash-fast_1: shared = {'mash_output': '_output'}]
# Compute data-driven / canonical prior matrices (time estimate: ~20min)
depends: R_library("ExtremeDeconvolution"), R_library("mashr@stephenslab/mashr"), flash_data, sfa_data
parameter: alpha = 1
K = 3 # as in mash paper
P = 3 # as in mash paper
input: f"{data:a}"
output: f"{cwd:a}/{data:bn}.V{vhat}.K{K}.P{P}.rds"
R: expand = "${ }", workdir = cwd
    dat = readRDS(${_input:r})
    null.z = rbind(dat$train.z, dat$validate.z)
    if (${vhat}) {
        V = as.matrix(cor(null.z[which(apply(abs(null.z),1, max) < 2),]))
    } else {
        V = as.matrix(diag(ncol(null.z)))
    }
    mash_data = mashr::mash_set_data(as.matrix(dat$test.b), Shat=as.matrix(dat$test.s), alpha=${alpha}, V=V)
    sfa_data = readRDS(${sfa_data:r})
    sfa_res = as.matrix(sfa_data$lambda) %*% as.matrix(sfa_data$F)
    # SFA matrices
    U.sfa = c(mashr::cov_from_factors(as.matrix(sfa_data$F), "sfa${K}"), list("tSFA" = t(sfa_res) %*% sfa_res / nrow(dat$test.z)))
    # FLASH matrices
    flash_res = readRDS(${flash_data:r})
    # SVD matrices
    U.pca = mashr::cov_pca(mash_data, ${P})
    # D.center is used to compute emperical cov matrix
    D.center = apply(as.matrix(dat$test.z), 2, function(x) x - mean(x))
    # Denoised data-driven matrices
    U.dd = c(U.sfa, flash_res$U.flash, flash_res$U.loading, U.pca, list("XX" = t(D.center) %*% D.center / nrow(dat$test.z)))
    U.ed = mashr::cov_ed(mash_data, U.dd)
    # Canonical matrices
    U.can = mashr::cov_canonical(mash_data)
    saveRDS(list(Ulist = c(U.ed, U.can), DD_raw = U.dd, V = V), ${_output:r})

Workflow can only be executed with magic %run or %sosrun.

In [9]:
[mash-fast_2]
# Fit MASH mixture model (time estimate: <5min)
depends: R_library("REBayes")
output: f"{_input:n}.mash_model.rds"
R: expand = "${ }", workdir = cwd, env = {'MOSEKLM_LICENSE_FILE': str(mosek_license)}
    dat = readRDS(${data:ar})
    settings = readRDS(${_input:r})
    train.b = rbind(dat$train.b, dat$validate.b)
    train.s = rbind(dat$train.s, dat$validate.s)
    mash_data = mashr::mash_set_data(as.matrix(dat$train.b), Shat=as.matrix(dat$train.s), alpha=${alpha}, V=settings$V)
    saveRDS(mashr::mash(mash_data, Ulist = settings$Ulist, outputlevel = 1), ${_output:r})

Workflow can only be executed with magic %run or %sosrun.

In [10]:
[mash-fast_3]
# Posterior inference on the "top" set of gene-snp pairs (time estimate: <3min)
output: f"{_input:n}.posterior.rds"
R: expand = "${ }", workdir = cwd
    dat = readRDS(${data:ar})
    settings = readRDS(${mash_output:r})
    mash_data = mashr::mash_set_data(as.matrix(dat$test.b), Shat=as.matrix(dat$test.s), alpha=${alpha}, V=settings$V)
    saveRDS(mashr::mash_compute_posterior_matrices(readRDS(${_input:r}), mash_data), ${_output:r})

Workflow can only be executed with magic %run or %sosrun.

## Export and run default pipeline

In [11]:
[export]
# Export notebook to HTML file
input: [item for item in paths(sys.argv) if item.suffix == '.ipynb'], group_by = 1
output: [(f'{cwd:a}/{item:bn}.full.html', f'{cwd:a}/{item:bn}.lite.html') for item in paths(sys.argv) if item.suffix == '.ipynb'], group_by = 2
bash: expand = True, stderr = False
  sos convert {_input} {_output[0]}
  sos convert {_input} {_output[1]} --template sos-report
  
[default]
# Run all analysis in this notebook
sos_run('export')
if algorithm == 'paper':
    sos_run('mash-paper')
else:
    fail_if(not mosek_license.is_file(), msg = f'Please put a valid copy (NOT a symbolic link!) of MOSEK license to: \n``{mosek_license}``')
    sos_run('mash-fast')

Workflow can only be executed with magic %run or %sosrun.

In [12]:
%sessioninfo

SoS Version,0.9.14.2
